# Elastic net (MADlib v1.10+)
Demonstrates elastic net, including these updates:
- in MADlib 1.10: grouping and cross validation introduced 
- in MADlib 1.13: report negative root mean squared error instead of the negative mean squared error

In [5]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [13]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.15-dev, git revision: rc/1.14-rc1-23-gabafa66, cmake configuration time: Wed Jul 11 00:36:05 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


## 1.  Create data set
House prices and characteristics.

In [4]:
%%sql 
DROP TABLE IF EXISTS houses;

CREATE TABLE houses ( id INT,
                      tax INT,
                      bedroom INT,
                      bath FLOAT,
                      price INT,
                      size INT,
                      lot INT,
                      zipcode INT);

INSERT INTO houses (id, tax, bedroom, bath, price, size, lot, zipcode) VALUES
(1  ,  590 ,       2 ,    1 ,  50000 ,  770 , 22100  , 94301),
(2  , 1050 ,       3 ,    2 ,  85000 , 1410 , 12000  , 94301),
(3  ,   20 ,       3 ,    1 ,  22500 , 1060 ,  3500  , 94301),
(4  ,  870 ,       2 ,    2 ,  90000 , 1300 , 17500  , 94301),
(5  , 1320 ,       3 ,    2 , 133000 , 1500 , 30000  , 94301),
(6  , 1350 ,       2 ,    1 ,  90500 ,  820 , 25700  , 94301),
(7  , 2790 ,       3 ,  2.5 , 260000 , 2130 , 25000  , 94301),
(8  ,  680 ,       2 ,    1 , 142500 , 1170 , 22000  , 94301),
(9  , 1840 ,       3 ,    2 , 160000 , 1500 , 19000  , 94301),
(10 , 3680 ,       4 ,    2 , 240000 , 2790 , 20000  , 94301),
(11 , 1660 ,       3 ,    1 ,  87000 , 1030 , 17500  , 94301),
(12 , 1620 ,       3 ,    2 , 118600 , 1250 , 20000  , 94301),
(13 , 3100 ,       3 ,    2 , 140000 , 1760 , 38000  , 94301),
(14 , 2070 ,       2 ,    3 , 148000 , 1550 , 14000  , 94301),
(15 ,  650 ,       3 ,  1.5 ,  65000 , 1450 , 12000  , 94301),
(16 ,  770 ,       2 ,    2 ,  91000 , 1300 , 17500  , 76010),
(17 , 1220 ,       3 ,    2 , 132300 , 1500 , 30000  , 76010),
(18 , 1150 ,       2 ,    1 ,  91100 ,  820 , 25700  , 76010),
(19 , 2690 ,       3 ,  2.5 , 260011 , 2130 , 25000  , 76010),
(20 ,  780 ,       2 ,    1 , 141800 , 1170 , 22000  , 76010),
(21 , 1910 ,       3 ,    2 , 160900 , 1500 , 19000  , 76010),
(22 , 3600 ,       4 ,    2 , 239000 , 2790 , 20000  , 76010),
(23 , 1600 ,       3 ,    1 ,  81010 , 1030 , 17500  , 76010),
(24 , 1590 ,       3 ,    2 , 117910 , 1250 , 20000  , 76010),
(25 , 3200 ,       3 ,    2 , 141100 , 1760 , 38000  , 76010),
(26 , 2270 ,       2 ,    3 , 148011 , 1550 , 14000  , 76010),
(27 ,  750 ,       3 ,  1.5 ,  66000 , 1450 , 12000  , 76010);

SELECT * FROM houses ORDER BY id;

Done.
Done.
27 rows affected.
27 rows affected.


id,tax,bedroom,bath,price,size,lot,zipcode
1,590,2,1.0,50000,770,22100,94301
2,1050,3,2.0,85000,1410,12000,94301
3,20,3,1.0,22500,1060,3500,94301
4,870,2,2.0,90000,1300,17500,94301
5,1320,3,2.0,133000,1500,30000,94301
6,1350,2,1.0,90500,820,25700,94301
7,2790,3,2.5,260000,2130,25000,94301
8,680,2,1.0,142500,1170,22000,94301
9,1840,3,2.0,160000,1500,19000,94301
10,3680,4,2.0,240000,2790,20000,94301


## 2. Train the model

In [5]:
%%sql
DROP TABLE IF EXISTS houses_en, houses_en_summary;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en',               -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 0.5,                       -- Alpha value
                                 0.1,                       -- Lambda value
                                 TRUE,                      -- Standardize
                                 NULL,                      -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 '',                        -- Optimizer parameters
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en;

Done.
1 rows affected.
1 rows affected.


family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'bath', u'size']","[22.7851318679, 10707.9553682, 54.7961166559]","[22.7851318679, 10707.9553682, 54.7961166559]",-7798.78310728,-512248641.97,True,10000


# 3. Prediction
Evaluate residuals.

In [6]:
%%sql
SELECT id, price, predict, price - predict AS residual
FROM (
    SELECT
        houses.*,
        madlib.elastic_net_gaussian_predict(
            m.coef_all,              -- Coefficients
            m.intercept,             -- Intercept
            ARRAY[tax,bath,size]     -- Features (corresponding to coefficients)
            ) AS predict
    FROM houses, houses_en m) s
ORDER BY id;

27 rows affected.


id,price,predict,residual
1,50000,58545.409888,-8545.40988802
2,85000,114804.040575,-29804.0405752
3,22500,61448.7585535,-38948.7585535
4,90000,104675.144007,-14675.1440069
5,133000,125887.676679,7112.3233214
6,90500,78601.9159404,11898.0840596
7,260000,199257.351702,60742.6482983
8,142500,82514.5184185,59985.4815815
9,160000,137735.94525,22264.0547501
10,240000,250347.578373,-10347.578373


# 4. Grouping 
Group on zip code.

In [8]:
%%sql
DROP TABLE IF EXISTS houses_en1, houses_en1_summary;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en1',              -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 0.5,                       -- Alpha value
                                 0.1,                       -- Lambda value
                                 TRUE,                      -- Standardize
                                 'zipcode',                 -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 '',                        -- Optimizer parameters
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en1;

Done.
1 rows affected.
2 rows affected.


zipcode,family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
76010,gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'bath', u'size']","[14.9802020928, 9133.17041265, 62.8225614522]","[14.9802020928, 9133.17041265, 62.8225614522]",14.7294468096,-525667117.987,True,10000
94301,gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'bath', u'size']","[27.6945649037, 11509.010807, 49.0945476263]","[27.6945649037, 11509.010807, 49.0945476263]",-11145.5017384,-520358795.785,True,10000


Prediction function

In [9]:
%%sql
SELECT madlib.elastic_net_predict(
                'houses_en1',             -- Model table
                'houses',                 -- New source data table
                'id',                     -- Unique ID associated with each row
                'houses_en1_prediction'   -- Table to store prediction result
              );

SELECT  houses.id,
        houses.price,
        houses_en1_prediction.prediction,
        houses.price - houses_en1_prediction.prediction AS residual
FROM houses_en1_prediction, houses
WHERE houses.id = houses_en1_prediction.id ORDER BY id;

1 rows affected.
27 rows affected.


id,price,prediction,residual
1,50000,54506.104034,-4506.10403403
2,85000,110175.125178,-25175.1251776
3,22500,52957.6208506,-30457.6208506
4,90000,99789.703256,-9789.70325601
5,133000,122071.166988,10928.8330121
6,90500,78008.7007422,12491.2992578
7,260000,199466.247804,60533.7521956
8,142500,76636.4339259,65863.5660741
9,160000,136472.340738,23527.6592621
10,240000,250762.306599,-10762.3065986


## 5.  When coef_nonzero is different from coef_all
Train

In [10]:
%%sql
DROP TABLE IF EXISTS houses_en2, houses_en2_summary;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en2',              -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 1,                         -- Alpha value
                                 30000,                     -- Lambda value
                                 TRUE,                      -- Standardize
                                 NULL,                      -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 '',                        -- Optimizer parameters
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en2;

Done.
1 rows affected.
1 rows affected.


family,features,features_selected,coef_nonzero,coef_all,intercept,log_likelihood,standardize,iteration_run
gaussian,"[u'tax', u'bath', u'size']","[u'tax', u'size']","[6.94383308191, 29.7206857861]","[6.94383308191, 0.0, 29.7206857861]",74441.4573381,-1635348584.1,True,173


Prediction function with coef_all to evaluate residuals.

In [11]:
%%sql
SELECT id, price, predict, price - predict AS residual
FROM (
    SELECT
        houses.*,
        madlib.elastic_net_gaussian_predict(
            m.coef_all,                   -- All coefficients
            m.intercept,                  -- Intercept
            ARRAY[tax,bath,size]          -- All features
            ) AS predict
    FROM houses, houses_en2 m) s
ORDER BY id;

27 rows affected.


id,price,predict,residual
1,50000,101423.246912,-51423.2469117
2,85000,123638.649033,-38638.6490325
3,22500,106084.260933,-83584.260933
4,90000,119119.483641,-29119.4836413
5,133000,128188.345685,4811.65431463
6,90500,108186.594343,-17686.5943433
7,260000,157119.812361,102880.187639
8,142500,113936.466204,28563.5337965
9,160000,131799.138888,28200.861112
10,240000,182915.476423,57084.5235773


We can speed up the prediction function with coef_nonzero to evaluate residuals. This requires the user to examine the feature_selected column in the result table to construct the correct set of independent variables to provide to the prediction function.

In [12]:
%%sql
SELECT id, price, predict, price - predict AS residual
FROM (
    SELECT
        houses.*,
        madlib.elastic_net_gaussian_predict(
            m.coef_nonzero,               -- Non-zero coefficients
            m.intercept,                  -- Intercept
            ARRAY[tax,size]               -- Features corresponding to non-zero coefficients
            ) AS predict
    FROM houses, houses_en2 m) s
ORDER BY id;

27 rows affected.


id,price,predict,residual
1,50000,101423.246912,-51423.2469117
2,85000,123638.649033,-38638.6490325
3,22500,106084.260933,-83584.260933
4,90000,119119.483641,-29119.4836413
5,133000,128188.345685,4811.65431463
6,90500,108186.594343,-17686.5943433
7,260000,157119.812361,102880.187639
8,142500,113936.466204,28563.5337965
9,160000,131799.138888,28200.861112
10,240000,182915.476423,57084.5235773


## 6.  Cross validation
Reuse the houses table above. Here we use 3-fold cross validation with 3 automatically generated lambda values and 3 specified alpha values. (This can take some time to run since elastic net is effectively being called 27 times for these combinations, then a 28th time for the whole dataset.)

In [ ]:
%%sql
DROP TABLE IF EXISTS houses_en3, houses_en3_summary, houses_en3_cv;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en3',               -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 0.5,                       -- Alpha value
                                 0.1,                       -- Lambda value
                                 TRUE,                      -- Standardize
                                 NULL,                      -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 $$ n_folds = 3,            -- Optimizer parameters
                                    validation_result=houses_en3_cv,
                                    n_lambdas = 3, 
                                    alpha = {0, 0.1, 1}
                                 $$,                       
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en3;

Details of the cross validation:

In [9]:
%%sql
SELECT * FROM houses_en3_cv ORDER BY mean_neg_loss DESC;

9 rows affected.


alpha,lambda_value,mean_neg_loss,std_neg_loss
0.0,0.1,-36094.4685768,10524.4473253
0.1,0.1,-36136.2448004,10682.4136993
1.0,100.0,-37007.9496501,12679.3781975
1.0,0.1,-37018.1019927,12716.7438015
0.1,100.0,-59275.6940173,9764.50064237
0.0,100.0,-59380.252681,9763.26373034
1.0,100000.0,-60353.0220769,9748.10305107
0.1,100000.0,-143513752113000000000000000000000000000000000000000000,157073834312000000000000000000000000000000000000000000
0.0,100000.0,-11248884473800000000000000000000000000000000000000000000,9490568229990000000000000000000000000000000000000000000


In [12]:
%%sql
SELECT * FROM houses_en3_summary;

1 rows affected.


method,source_table,out_table,dependent_varname,independent_varname,family,alpha,lambda_value,grouping_col,num_all_groups,num_failed_groups
elastic_net,houses,houses_en3,price,"array[tax, bath, size]",gaussian,0.0,0.1,NULL,1,0


# 6a. Cross validation
Here we use 3-fold cross validation with 3 automatically generated lambda values and 1 alpha value (i.e., 9 times).

In [ ]:
%%sql
DROP TABLE IF EXISTS houses_en3, houses_en3_summary, houses_en3_cv;
SELECT madlib.elastic_net_train( 'houses',                  -- Source table
                                 'houses_en3',               -- Result table
                                 'price',                   -- Dependent variable
                                 'array[tax, bath, size]',  -- Independent variable
                                 'gaussian',                -- Regression family
                                 0.5,                       -- Alpha value
                                 0.1,                       -- Lambda value
                                 TRUE,                      -- Standardize
                                 NULL,                      -- Grouping column(s)
                                 'fista',                   -- Optimizer
                                 $$ n_folds = 3,            -- Optimizer parameters
                                    validation_result=houses_en3_cv,
                                    n_lambdas = 3
                                 $$,                       
                                 NULL,                      -- Excluded columns
                                 10000,                     -- Maximum iterations
                                 1e-6                       -- Tolerance value
                               );
SELECT * FROM houses_en3;

Details of the cross validation:

In [16]:
%%sql
SELECT * FROM houses_en3_cv ORDER BY mean_neg_loss DESC;

3 rows affected.


lambda_value,mean_neg_loss,std_neg_loss
100000.0,-255543791799000000000000000000000000000000000,442158712729000000000000000000000000000000000
100.0,-59332.2198813,8220.8755071
0.1,-51938.9613421,28946.523247


In [ ]:
%%sql
SELECT * FROM houses_en3_summary;